In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
df = pd.read_csv('train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.info

<bound method DataFrame.info of           id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2      

In [6]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [8]:
df['label'].nunique()

2

In [9]:
df = df.dropna()

In [10]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [11]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [12]:
print(X.shape)
print(y.shape)

(18285, 4)
(18285,)


In [13]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [14]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot


In [15]:
#vocabulary size
voc_size=5000

<h3> One Hot representation </h3>

In [16]:
message = X.copy()

In [17]:
message.reset_index(inplace=True)
#As we dropped some of the Nan values.

In [18]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\E0551793\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
ps=PorterStemmer()
corpus=[]
for i in range(len(message)):
    text = re.sub('[^a-zA-Z]', ' ', message['title'][i])
    text = text.lower()
    text = text.split()

    text = [ps.stem(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    corpus.append(text)

In [21]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [22]:
one_hot_rep = [one_hot(word, voc_size) for word in corpus]
one_hot_rep

[[4305, 3757, 627, 3835, 4344, 1291, 1032, 943, 601, 743],
 [1894, 2532, 3908, 3099, 728, 137, 788],
 [4056, 4525, 837, 438],
 [2133, 1823, 2357, 1679, 824, 766],
 [2624, 728, 848, 4024, 4654, 3377, 728, 4601, 1740, 1131],
 [3071,
  4197,
  2477,
  3603,
  2697,
  2133,
  284,
  1090,
  1003,
  3660,
  1811,
  819,
  676,
  3398,
  788],
 [1914, 535, 4098, 739, 1951, 1571, 3965, 102, 2083, 4019, 3898],
 [1459, 3773, 1317, 3574, 1464, 195, 2133, 2376, 2083, 4019, 3898],
 [466, 2026, 2553, 2295, 964, 662, 3149, 2283, 2133, 3556],
 [1992, 4845, 2895, 2693, 4227, 4397, 609, 1428],
 [2393, 2369, 4748, 959, 2187, 820, 1320, 4896, 4161, 4788, 2012],
 [1679, 2145, 4344, 662, 2133, 1464],
 [176, 3302, 4938, 1725, 311, 85, 4054, 4759, 4296],
 [869, 2467, 2519, 157, 4373, 1832, 160, 2083, 4019, 3898],
 [3838, 436, 2548, 2379, 4294, 2083, 4019, 3898],
 [1071, 3517, 2016, 1241, 119, 591, 1233, 4270, 2227, 3133],
 [3503, 3467, 2532],
 [4845, 4418, 335, 502, 2133, 3808, 2638, 788],
 [15, 3160, 3908, 

Padding

In [23]:
sent_len = 20
pad_docs = pad_sequences(one_hot_rep, padding='pre', maxlen=sent_len)
print(pad_docs)

[[   0    0    0 ...  943  601  743]
 [   0    0    0 ...  728  137  788]
 [   0    0    0 ... 4525  837  438]
 ...
 [   0    0    0 ... 2083 4019 3898]
 [   0    0    0 ... 3171  564 4274]
 [   0    0    0 ... 2411 2285 3558]]


In [24]:
len(pad_docs)

18285

In [25]:
feature_dim = 40
model = Sequential()
model.add(Embedding(voc_size, feature_dim, input_length = sent_len))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


c:\Users\E0551793\OneDrive - Eaton\BLC-Saloni\BITSPilani\FakeNewsClassifier-LSTM\venv\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [26]:
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [27]:
import numpy as np
X_final = np.array(pad_docs)
y_final = np.array(y)

In [28]:
len(pad_docs), y.shape

(18285, (18285,))

In [29]:
X_final.shape

(18285, 20)

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [31]:
y_test

array([1, 0, 0, ..., 0, 1, 1], dtype=int64)

In [32]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10


192/192 ━━━━━━━━━━━━━━━━━━━━ 18s 49ms/step - accuracy: 0.7626 - loss: 0.4498 - val_accuracy: 0.9090 - val_loss: 0.2115
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - accuracy: 0.9381 - loss: 0.1510 - val_accuracy: 0.9231 - val_loss: 0.1890
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.9633 - loss: 0.1003 - val_accuracy: 0.9218 - val_loss: 0.2028
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - accuracy: 0.9761 - loss: 0.0704 - val_accuracy: 0.9208 - val_loss: 0.2276
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - accuracy: 0.9846 - loss: 0.0488 - val_accuracy: 0.9143 - val_loss: 0.2259
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.9862 - loss: 0.0416 - val_accuracy: 0.9206 - val_loss: 0.3252
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.9933 - loss: 0.0225 - val_accuracy: 0.9130 - val_loss: 0.2840
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 49ms/step - accuracy: 0.9936 - loss: 0.0213 - val_accuracy: 0.91

In [34]:
# tf.keras.utils.plot_model(model)

Performance Metrics

In [35]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")
# y_pred = np.argmax(y_pred_value,axis=1)

from sklearn.metrics import confusion_matrix

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [36]:
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [37]:
confusion_matrix(y_test, y_pred)

array([[3129,  290],
       [ 241, 2375]], dtype=int64)

In [38]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.912013256006628

Make Predictions

In [39]:
test_df = pd.read_csv("test.csv")

In [40]:
test_df.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [41]:
test_df.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [42]:
test_df = test_df.dropna()

In [43]:
test_message = test_df.copy()
test_message.reset_index(inplace=True)

In [44]:
ps=PorterStemmer()
corpus=[]
for i in range(len(test_message)):
    text = re.sub('[^a-zA-Z]', ' ', test_message['title'][i])
    text = text.lower()
    text = text.split()

    text = [ps.stem(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    corpus.append(text)

In [45]:
corpus

['specter trump loosen tongu purs string silicon valley new york time',
 'nodapl nativ american leader vow stay winter file lawsuit polic',
 'tim tebow attempt anoth comeback time basebal new york time',
 'keiser report meme war e',
 'pelosi call fbi investig find russian donald trump breitbart',
 'weekli featur profil randi shannon',
 'u gener admir endors trump command chief',
 'work class hero john brennon',
 'rise mandatori vaccin mean end medic freedom',
 'communist terror small busi',
 'comput programm come forward admit paid rig vote booth trump right usa newsflash',
 'thiev take chunk chang pound berlin museum new york time',
 'new england patriot owner still sore n f l payback sight new york time',
 'colleg republican yaf sue berkeley ann coulter event breitbart',
 'trump melt accus us postal servic steal elect clinton',
 'visit madagascar leav red swimsuit lemur recip home new york time',
 'rees peanut butter cup cheap full toxic chemic',
 'presid obama presid elect donald tr

In [46]:
one_hot_rep = [one_hot(word, voc_size) for word in corpus]
one_hot_rep

[[1028, 2133, 4379, 3212, 4275, 3217, 2137, 1935, 2083, 4019, 3898],
 [4610, 3242, 56, 3661, 2391, 3268, 4684, 61, 2854, 3921],
 [2198, 795, 3804, 1291, 1890, 3898, 4962, 2083, 4019, 3898],
 [4062, 4504, 3917, 3872, 3453],
 [363, 717, 3503, 1663, 4373, 2393, 502, 2133, 788],
 [545, 3383, 3115, 4165, 4730],
 [350, 2916, 1667, 3065, 2133, 2034, 2548],
 [3964, 3058, 3410, 2076, 3800],
 [927, 3147, 2614, 3065, 504, 2101, 1263],
 [3729, 822, 3016, 4072],
 [1986, 1571, 4688, 953, 2073, 3302, 4189, 1431, 3860, 2133, 2164, 2404, 1970],
 [3013, 2203, 698, 4853, 3184, 1892, 4718, 2083, 4019, 3898],
 [2083,
  2058,
  2369,
  1597,
  1242,
  3264,
  1633,
  2285,
  1492,
  4079,
  3368,
  2083,
  4019,
  3898],
 [377, 3277, 2618, 3162, 908, 4816, 3182, 773, 788],
 [2133, 2090, 2129, 1679, 3247, 2396, 3979, 4987, 3908],
 [839, 2764, 3703, 3734, 3540, 2421, 468, 1832, 2083, 4019, 3898],
 [2795, 1774, 2040, 4254, 3310, 4112, 2850, 3013],
 [597, 466, 597, 4987, 502, 2133, 3361, 4586, 4305],
 [2693, 12

In [47]:
pad_docs = pad_sequences(one_hot_rep, padding='pre', maxlen=sent_len)
print(pad_docs)

[[   0    0    0 ... 2083 4019 3898]
 [   0    0    0 ...   61 2854 3921]
 [   0    0    0 ... 2083 4019 3898]
 ...
 [   0    0    0 ... 2083 4019 3898]
 [   0    0    0 ... 2083 4019 3898]
 [   0    0    0 ... 2083 4019 3898]]


In [48]:
X_test = np.array(pad_docs)

In [49]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [50]:
predictions = [int(np.round(pred[0])) for pred in y_pred]
predictions

[0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,


In [51]:
submission = pd.DataFrame({
    "id" : test_df['id'],
    "label" : predictions
})
submission

,id,label
0,20800,0
2,20802,0
3,20803,0
4,20804,1
6,20806,0
...,...,...
5194,25994,0
5195,25995,0
5196,25996,0
5197,25997,0


In [52]:
submission.to_csv('submission.csv', index=False)